In [1]:
#python packages pd
import numpy as np
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Activation
from keras.layers import CuDNNLSTM
from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from sklearn.model_selection import train_test_split
import importlib

import utilis

Using TensorFlow backend.


In [2]:
# Check that you are running GPU's
utilis.GPU_checker()





You are runnning an instance with 4 GPU's


In [3]:
utilis.aws_setup()

AWS SETUP SHOULD BE COMPLETE, we are on <botocore.client.S3 object at 0x7fcee471b7b8>


# Model bi directional LSTM without hypotheses

In [4]:
sys.path.append('..')

In [23]:
from packages import generator

In [6]:
# generator config
W_HYP = False

In [24]:
%%time
#  generators
importlib.reload(generator)
training_generator = generator.Keras_DataGenerator(data_dir='',  dataset='train_new', w_hyp=W_HYP)
validation_generator = generator.Keras_DataGenerator(data_dir='', dataset='valid_new', w_hyp= W_HYP)

# of batches:  5888.0
Generating examples from a set of 376832 examples 

# of batches:  1600.0
Generating examples from a set of 102400 examples 

CPU times: user 2.17 ms, sys: 555 µs, total: 2.72 ms
Wall time: 1.9 ms


In [8]:
#Constants
# ARE YOU LOADINNG MODE?
VOCAB_SIZE = 1254
INPUT_LENGTH = 1000
EMBEDDING_DIM = 512


# constnats
INPUT_LENGTH = 3000 if W_HYP else 1000
print(INPUT_LENGTH)


# model
def build_model(vocab_size, embedding_dim, input_length):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=input_length))
    model.add(SpatialDropout1D(0.4))
    model.add(Bidirectional(CuDNNLSTM(512)))
    model.add(Dropout(0.4))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(41, activation='softmax'))
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

1000

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 512)         642048    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1000, 512)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1024)              4202496   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
____________________

In [13]:
## ARE YOU LOADING A MODEL IF YES RUN TEH FOLLOWING LINES 
# from keras.models import model_from_json
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# # REMEMEBER TO COMPILE 
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
#overwriting model
# model = loaded_model

In [29]:
%%time
#try and make it run until 9 am GMT+1
n_epochs = 10
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,
                            verbose=1,
                            use_multiprocessing=True,
                            epochs=n_epochs)

Epoch 1/10
500/500 [==============================] - 298s 595ms/step - loss: 1.6557 - acc: 0.4741 - val_loss: 3.3538 - val_acc: 0.2519
Epoch 2/10
500/500 [==============================] - 292s 585ms/step - loss: 1.4299 - acc: 0.5333 - val_loss: 3.5456 - val_acc: 0.2473
Epoch 3/10
500/500 [==============================] - 294s 588ms/step - loss: 1.1165 - acc: 0.6290 - val_loss: 4.0802 - val_acc: 0.2380
Epoch 4/10
500/500 [==============================] - 294s 587ms/step - loss: 0.7896 - acc: 0.7416 - val_loss: 5.1524 - val_acc: 0.2456
Epoch 5/10
500/500 [==============================] - 293s 586ms/step - loss: 0.5482 - acc: 0.8290 - val_loss: 5.8332 - val_acc: 0.2385
Epoch 6/10
500/500 [==============================] - 294s 589ms/step - loss: 0.3968 - acc: 0.8807 - val_loss: 5.9636 - val_acc: 0.2402
Epoch 7/10
500/500 [==============================] - 292s 585ms/step - loss: 0.2993 - acc: 0.9078 - val_loss: 6.3585 - val_acc: 0.2219
Epoch 8/10
500/500 [============================

In [30]:
# FOR SAVING MODEL
model_json = model.to_json()
with open("model_seriously_overfitted.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_seriously_overfitted.h5")
print("Saved model to disk")

Saved model to disk


In [31]:
#WARNING_DECIDE_HOW_TO_NAME_LOG
#descriptionofmodel_personwhostartsrun
#e.g. LSTM_128encoder_etc_tanc
LOSS_FILE_NAME = "forjeff4"

#WARNING NUMBER 2 - CURRENTLY EVERYTIME YOU RERUN THE CELLS BELOW THE FILES WITH THOSE NAMES GET WRITTEN OVER

In [32]:
# save history - WARNING FILE NAME 
utilis.history_saver_bad(history, LOSS_FILE_NAME)

SAVED SOME LOGS -- OVERWROTE OLD LOGS -- SOMEONE NEEDS TO FIX THIS
